# Dates pivot table

In [1]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np

/home/ednilson/.virtualenvs/jupyter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Get collection information from ArticleMeta 

In [2]:
AMC_URL = "http://articlemeta.scielo.org/api/v1/collection/identifiers/"
amc_data = pd.DataFrame(json.load(urlopen(AMC_URL)))

In [3]:
amc_data.head(6)

,acron,acron2,code,document_count,domain,has_analytics,is_active,journal_count,name,original_name,status,type
0,arg,ar,arg,39006.0,www.scielo.org.ar,True,True,"{'deceased': 22, 'current': 125}","{'en': 'Argentina', 'pt': 'Argentina', 'es': '...",Argentina,certified,journals
1,chl,cl,chl,63467.0,www.scielo.cl,True,True,"{'deceased': 13, 'suspended': 1, 'current': 105}","{'en': 'Chile', 'pt': 'Chile', 'es': 'Chile'}",Chile,certified,journals
2,col,co,col,70213.0,www.scielo.org.co,True,True,"{'suspended': 7, 'current': 226}","{'en': 'Colombia', 'pt': 'Colombia', 'es': 'Co...",Colombia,certified,journals
3,cub,cu,cub,33492.0,scielo.sld.cu,True,True,"{'deceased': 2, 'suspended': 4, 'current': 61}","{'en': 'Cuba', 'pt': 'Cuba', 'es': 'Cuba'}",Cuba,certified,journals
4,esp,es,esp,37946.0,scielo.isciii.es,True,True,"{'deceased': 6, 'suspended': 11, 'current': 43}","{'en': 'Spain', 'pt': 'Espanha', 'es': 'España'}",España,certified,journals
5,mex,mx,mex,64406.0,www.scielo.org.mx,True,True,"{'deceased': 12, 'suspended': 44, 'current': 159}","{'en': 'Mexico', 'pt': 'Mexico', 'es': 'Mexico'}",Mexico,certified,journals


Some collections won't be analyzed, mainly to avoid duplicates
(there are articles in more than one collection).
The `spa` (*Public Health* collection) should have part of it
kept in the result, but it's not a collection
whose journals/articles are assigned to a single country.
The collections below are linked to a single country:

In [4]:
dont_evaluate = ["bio", "cci", "cic", "ecu", "psi", "pry", "rve", "rvo", "rvt", "sss", "spa", "wid"]
amc_names_map = {
    "code": "collection",
    "acron2": "origin",
}
amc_pairs = amc_data \
    [(amc_data["acron2"].str.len() == 2) &
     ~amc_data["code"].isin(dont_evaluate)] \
    [list(amc_names_map.keys())] \
    .rename(columns=amc_names_map) \
    .assign(origin=lambda df: df["origin"].str.upper())
amc_pairs

,collection,origin
0,arg,AR
1,chl,CL
2,col,CO
3,cub,CU
4,esp,ES
5,mex,MX
6,prt,PT
8,scl,BR
11,sza,ZA
12,ven,VE


## ISSN selection from `spa`

These journals in the `spa` collection have the following countries:

In [5]:
spa_issn_country = pd.DataFrame([
    ("0021-2571", "IT"),
    ("0042-9686", "CH"),
    ("1020-4989", "US"),
    ("1555-7960", "US"),
], columns=["issn", "origin"])
spa_issn_country # For collection = "spa", only!

,issn,origin
0,0021-2571,IT
1,0042-9686,CH
2,1020-4989,US
3,1555-7960,US


## Languages dataset

This dataset is the
[Network spreadsheet/CSV pack](https://static.scielo.org/tabs/tabs_network.zip)
 which can be found in the
[SciELO Analytics report](https://analytics.scielo.org/w/reports)
web page.
The first two rows of it are:

#### Unzip the CSV file

In [10]:
import zipfile
# Use the Zip file in jcatalog/data/scielo
with zipfile.ZipFile('../../data/scielo/tabs_network_190210.zip', 'r') as zip_ref:
    zip_ref.extract('documents_dates.csv', 'csv_files')

In [11]:
dataset = pd.read_csv("csv_files/documents_dates.csv", keep_default_na=False)
dataset.head(3).T

/home/ednilson/.virtualenvs/jupyter/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2
extraction date,2019-02-10,2019-02-10,2019-02-10
study unit,document,document,document
collection,scl,scl,scl
ISSN SciELO,0100-879X,0100-879X,0100-879X
ISSN's,0100-879X;1414-431X,0100-879X;1414-431X,0100-879X;1414-431X
title at SciELO,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
title thematic areas,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences
title is agricultural sciences,0,0,0
title is applied social sciences,0,0,0
title is biological sciences,1,1,1


#### Simplify the column names

In [12]:
names_map = {
    "ISSN SciELO": "issn",
    "title at SciELO":"title",
    "document publishing ID (PID SciELO)": "pid",
    "document type":"type",
    "document is citable": "is_citable",
    "document publishing year": "year"
}
df = dataset[list(names_map.keys())].rename(columns=names_map)
df[:5].T

,0,1,2,3,4
issn,0100-879X,0100-879X,0100-879X,0100-879X,0100-879X
title,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
pid,S0100-879X1998000800006,S0100-879X1998000800011,S0100-879X1998000800005,S0100-879X1998000800009,S0100-879X1998000800010
type,research-article,rapid-communication,research-article,rapid-communication,rapid-communication
is_citable,1,1,1,1,1
year,1998,1998,1998,1998,1998


#### Add pub_year (ate_1996)

In [13]:
df["pub_year"] = np.where(df['year'] <= 1996, 'ate_1996', df["year"])

#### Add review type

In [16]:
df["tipo_review"] = np.where(df['type'] == "review-article", 1, 0)

In [17]:
df2 = df.rename(columns={"pid":"docs"})

In [18]:
df2.columns

Index(['issn', 'title', 'docs', 'type', 'is_citable', 'year', 'pub_year',
       'tipo_review'],
      dtype='object')

In [19]:
values_list = [
    "docs",
    "is_citable", 
    "tipo_review"]

td = df2.pivot_table(
     index=["issn"],
     values=values_list,
     columns=["pub_year"],
     aggfunc=np.count_nonzero,
     fill_value=0)
td['docs'][:11].T

issn,0001-3714,0001-3765,0001-6002,0001-6365,0002-0591,0002-192X,0002-7014,0003-2573,0004-0592,0004-0614,0004-0622
pub_year,,,,,,,,,,,
1997,0,0,0,0,0,0,0,0,0,0,0
1998,33,0,0,0,0,0,0,0,0,0,0
1999,64,0,0,41,0,0,0,0,0,0,0
2000,0,119,31,50,0,0,0,0,0,0,61
2001,0,74,35,48,0,0,0,0,0,0,70
2002,0,104,31,60,0,18,0,0,0,0,54
2003,0,41,41,51,0,29,0,0,0,0,60
2004,0,85,48,49,0,27,0,34,0,0,64
2005,0,52,44,57,0,29,56,79,0,175,50


In [20]:
td.columns.levels

FrozenList([['docs', 'is_citable', 'tipo_review'], ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 'ate_1996']])

### Renames the labels for CSV

In [21]:
td.keys()
for k in td.keys():
    print(k)

('docs', '1997')
('docs', '1998')
('docs', '1999')
('docs', '2000')
('docs', '2001')
('docs', '2002')
('docs', '2003')
('docs', '2004')
('docs', '2005')
('docs', '2006')
('docs', '2007')
('docs', '2008')
('docs', '2009')
('docs', '2010')
('docs', '2011')
('docs', '2012')
('docs', '2013')
('docs', '2014')
('docs', '2015')
('docs', '2016')
('docs', '2017')
('docs', '2018')
('docs', '2019')
('docs', 'ate_1996')
('is_citable', '1997')
('is_citable', '1998')
('is_citable', '1999')
('is_citable', '2000')
('is_citable', '2001')
('is_citable', '2002')
('is_citable', '2003')
('is_citable', '2004')
('is_citable', '2005')
('is_citable', '2006')
('is_citable', '2007')
('is_citable', '2008')
('is_citable', '2009')
('is_citable', '2010')
('is_citable', '2011')
('is_citable', '2012')
('is_citable', '2013')
('is_citable', '2014')
('is_citable', '2015')
('is_citable', '2016')
('is_citable', '2017')
('is_citable', '2018')
('is_citable', '2019')
('is_citable', 'ate_1996')
('tipo_review', '1997')
('tipo_r

In [22]:
newlabel = []
for k in td.keys():
    newlabel.append(k[0]+'_'+k[1])

In [23]:
newlabel

['docs_1997',
 'docs_1998',
 'docs_1999',
 'docs_2000',
 'docs_2001',
 'docs_2002',
 'docs_2003',
 'docs_2004',
 'docs_2005',
 'docs_2006',
 'docs_2007',
 'docs_2008',
 'docs_2009',
 'docs_2010',
 'docs_2011',
 'docs_2012',
 'docs_2013',
 'docs_2014',
 'docs_2015',
 'docs_2016',
 'docs_2017',
 'docs_2018',
 'docs_2019',
 'docs_ate_1996',
 'is_citable_1997',
 'is_citable_1998',
 'is_citable_1999',
 'is_citable_2000',
 'is_citable_2001',
 'is_citable_2002',
 'is_citable_2003',
 'is_citable_2004',
 'is_citable_2005',
 'is_citable_2006',
 'is_citable_2007',
 'is_citable_2008',
 'is_citable_2009',
 'is_citable_2010',
 'is_citable_2011',
 'is_citable_2012',
 'is_citable_2013',
 'is_citable_2014',
 'is_citable_2015',
 'is_citable_2016',
 'is_citable_2017',
 'is_citable_2018',
 'is_citable_2019',
 'is_citable_ate_1996',
 'tipo_review_1997',
 'tipo_review_1998',
 'tipo_review_1999',
 'tipo_review_2000',
 'tipo_review_2001',
 'tipo_review_2002',
 'tipo_review_2003',
 'tipo_review_2004',
 'tipo_r

In [24]:
newlabel[::24]

['docs_1997', 'is_citable_1997', 'tipo_review_1997']

In [25]:
td.columns = newlabel

In [26]:
td.T

issn,0001-3714,0001-3765,0001-6002,0001-6365,0002-0591,0002-192X,0002-7014,0003-2573,0004-0592,0004-0614,...,2518-4431,2520-9868,2526-8910,2531-0488,2531-1379,2545-7756,2594-1321,2594-9985,2595-3192,2619-6573
docs_1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_1998,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_1999,64,0,0,41,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2000,0,119,31,50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2001,0,74,35,48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2002,0,104,31,60,0,18,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2003,0,41,41,51,0,29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2004,0,85,48,49,0,27,0,34,0,0,...,0,0,0,0,0,0,0,0,0,0
docs_2005,0,52,44,57,0,29,56,79,0,175,...,0,0,0,0,0,0,0,0,0,0
docs_2006,0,65,48,74,0,51,57,74,0,185,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# td.to_csv("output/td_documents_dates_network.csv")